In [1]:
# libraries 
import pandas as pd 
import os
import numpy as np
import time
import random
from csv import reader
import time

# plots:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# scraping
from bs4 import BeautifulSoup
from lxml import etree #for using XPATH with beautifulsoup
import requests
import numpy as np

#JSON
import json

# GeoJSON
# import geopandas as gpd

# regular expression
import re 

# concurrent futures - boosts the process of scraping utilicing the CPU better
# two main classes- 
#       1) executor class: manages all the threads and workload
#       2) futures class: creates a little instance and manages data coming back
import concurrent.futures

# libraries: 
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ------------------------------------ the plan ----------------------------------


## 2 steps : 

 ## ---- Step 1: 
 ###  run requests for all 98 municipalities and collect the total hits then construct a list of nececarry parameters necessary to get the total hits below or equal to 10.000 hits 
 
### the parameter list will contain "municipal", "year", "housing_type" , "property size","sale price range"

### these parameters will be static 'registrationTypes': 'normal', 'sold': 'true', 'sortAscending': 'false',     'sortBy': 'soldDate
 
 
## ---- Step 2:
### Run the scraper using the parameter list 


# ------------------------------------------------------------------------------------

# ___________________________ [Step 1:] ________________________ 

# Setting up the User-agent and Header methods 

get_user_agent() - gives a random user-agent from a pre-defined user-agent list 

get_header() - calls the User-agent and constructs and returns a header 

In [29]:
# change user agent for each request randomly
def get_user_agent():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.57'
    ]
    user_agent = user_agent_list[random.randint(0,len(user_agent_list)-1)]
   
    return user_agent

def get_header():
    
    # get user-agent
    user_agent = get_user_agent()

     # set up header     
    header = {
        'authority': 'api.boligsiden.dk',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'origin': 'https://www.boligsiden.dk',
        'referer': 'https://www.boligsiden.dk/',
        'sec-ch-ua': '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': user_agent,
        'x-api-key': 'GWD0fljZzkc8GOLV',
    }
    return header 

In [64]:
################################################################
#                    municipalities list 
################################################################

municipalities_list = [
    'København',
    'Aarhus',
    'Aalborg',
    'Odense',
    'Vejle',
    'Esbjerg',
    'Frederiksberg',
    'Randers',
    'Silkeborg',
    'Viborg',
    'Horsens',
    'Kolding',
    'Roskilde',
    'Herning',
    'Næstved',
    'Slagelse',
    'Gentofte',
    'Sønderborg',
    'Holbæk',
    'Gladsaxe',
    'Skanderborg',
    'Hjørring',
    'Helsingør',
    'Køge',
    'Guldborgsund',
    'Svendborg',
    'Aabenraa',
    'Holstebro',
    'Frederikshavn',
    'Lyngby-Taarbæk',
    'Rudersdal',
    'Ringkøbing-Skjern',
    'Haderslev',
    'Høje-Taastrup',
    'Hillerød',
    'Hvidovre',
    'Faaborg-Midtfyn',
    'Fredericia',
    'Greve',
    'Ballerup',
    'Varde',
    'Favrskov',
    'Kalundborg',
    'Hedensted',
    'Frederikssund',
    'Vordingborg',
    'Egedal',
    'Skive',
    'Syddjurs',
    'Thisted',
    'Tårnby',
    'Vejen',
    'Rødovre',
    'Ikast-Brande',
    'Furesø',
    'Mariagerfjord',
    'Fredensborg',
    'Gribskov',
    'Assens',
    'Middelfart',
    'Lolland',
    'Bornholm',
    'Jammerbugt',
    'Faxe',
    'Brøndby',
    'Norddjurs',
    'Tønder',
    'Brønderslev',
    'Vesthimmerlands',
    'Ringsted',
    'Odsherred',
    'Nyborg',
    'Halsnæs',
    'Rebild',
    'Sorø',
    'Nordfyns',
    'Herlev',
    'Lejre',
    'Albertslund',
    'Billund',
    'Allerød',
    'Hørsholm',
    'Solrød',
    'Kerteminde',
    'Stevns',
    'Glostrup',
    'Odder',
    'Ishøj',
    'Struer',
    'Morsø',
    'Lemvig',
    'Vallensbæk',
    'Dragør',
    'Langeland',
    'Ærø',
    'Samsø',
    'Fanø',
    'Læsø'
]



In [55]:
def make_request(session, parameters, main_header):
    
    # make request
    response = session.get('https://api.boligsiden.dk/search/addresses', params=parameters, headers=main_header)

    # get total hits
    totalhit = response.json()['totalHits']
#     print(f'------------Total hits: {totalhit}')

    return totalhit
    

In [65]:
def check_hits(municipal, parameter_housingType, parameter_year,parameter_price, parameter_size,session):
    
    global parameter_list
    
    # -------------------------------------------
    # - Total hits : Municipality + housing type!
    # -------------------------------------------
    
    print()
    print('=====================================================================')
    print(f'----------------------  {municipal}  ---------------------------')
    print('=====================================================================')
    print()
    
    for house_category in parameter_housingType:
        
        print('-----------------------------------')
        print(f'+ Municipality: {municipal}')
        print(f'+ Housing type: {house_category}')
        print('-----------------------------------')
        
        par_municipal = municipal
        par_house_type = house_category
        par_year = "nothing"
        par_price_min = "nothing"
        par_price_max = "nothing"
        par_size_min = "nothing"
        par_size_max = "nothing"
        
        #----- set parameter
        parameters = {
            'municipalities': str(municipal),
            'per_page': '20', # ----------------------- static 
            'page': '1', #----------------------------- static 
            'registrationTypes': 'normal', # ---------- Static
            'addressTypes': str(par_house_type),
            'sold': 'true', # -------------------------- Static
            'sortAscending': 'false', # --------------- Static
            'sortBy': 'soldDate' # -------------------- Static
        }
        
        #----- get header 
        main_header = get_header()
        
        #----- make HTTPS Request 
        totalhits = make_request(session, parameters, main_header)
        
        if(totalhits<10000):
            print('----------------------------------- BINGO')
            
            temp_tuple = (par_municipal, par_house_type, par_year, par_price_min, par_price_max, par_size_min, par_size_max,totalhits)
            print("------ printing the tuble:")
            print(f'Municipal: {temp_tuple[0]}')
            print(f'par_house_type: {temp_tuple[1]}')
            print(f'par_year: {temp_tuple[2]}')
            print(f'par_price_min: {temp_tuple[3]}')
            print(f'par_price_max: {temp_tuple[4]}')
            print(f'par_size_min: {temp_tuple[5]}')
            print(f'par_size_max: {temp_tuple[6]}')
            print(f'TOTALHITS: {temp_tuple[7]}')
            
            parameter_list.append(temp_tuple)
#             return (temp_tuple)
        
        else:
            # ---------------------------------------------------
            # - Total hits : Municipality + housing type + year!
            # ----------------------------------------------------
            
            for year in parameter_year:
                
                print('-----------------------------------')
                print(f'+ Municipality: {municipal}')
                print(f'+ Housing type: {house_category}')
                print(f'+ Year: {year}')
                print('-----------------------------------')
                
                # --- set parameter for year
                par_year = year


                #----- set parameter
                parameters = {
                    'municipalities': str(municipal),
                    'per_page': '20', # ----------------------- static 
                    'page': '1', #----------------------------- static
                    'yearSoldFrom': str(year),
                    'yearSoldTo': str(year),
                    'registrationTypes': 'normal', # ---------- Static
                    'addressTypes': str(par_house_type),
                    'sold': 'true', # -------------------------- Static
                    'sortAscending': 'false', # --------------- Static
                    'sortBy': 'soldDate' # -------------------- Static
                }

                #----- get header 
                main_header = get_header()

                #----- make HTTPS Request 
                totalhits = make_request(session, parameters, main_header)

                if(totalhits<10000):
                    print('----------------------------------- BINGO')
                    
                    temp_tuple = (par_municipal, par_house_type, par_year, par_price_min, par_price_max, par_size_min, par_size_max, totalhits)
                    print("------ printing the tuble:")
                    print(f'Municipal: {temp_tuple[0]}')
                    print(f'par_house_type: {temp_tuple[1]}')
                    print(f'par_year: {temp_tuple[2]}')
                    print(f'par_price_min: {temp_tuple[3]}')
                    print(f'par_price_max: {temp_tuple[4]}')
                    print(f'par_size_min: {temp_tuple[5]}')
                    print(f'par_size_max: {temp_tuple[6]}')
                    print(f'TOTALHITS: {temp_tuple[7]}')
                    
                    parameter_list.append(temp_tuple)
#                     return (temp_tuple)
                
                else:
                    # ----------------------------------------------------------
                    # - Total hits : Municipality + housing type + year + Price!
                    # ----------------------------------------------------------

                    for price_index in range(0,len(parameter_price)):
                        
                        # ----- set price min and max parameters
                    
                        if(parameter_price[price_index] == 24900000): 
                            par_price_min = parameter_price[price_index]
                            par_price_max = 'infinity'
                        else:
                            par_price_min = parameter_price[price_index]
                            par_price_max = parameter_price[price_index+1]

                            
                        print('-----------------------------------')
                        print(f'+ Municipality: {municipal}')
                        print(f'+ Housing type: {house_category}')
                        print(f'+ Year: {year}')
                        print(f'+ price minimum: {par_price_min}')
                        print(f'+ price minimum: {par_price_max}')
                        print('-----------------------------------')


                        #----- set parameter
                        if par_price_max == 'infinity': 
                            parameters = {
                                'municipalities': str(municipal),
                                'per_page': '20', # ----------------------- static 
                                'page': '1', #----------------------------- static
#                                 'priceMax': str(par_price_max),
                                'priceMin': str(par_price_min),
                                'yearSoldFrom': str(year),
                                'yearSoldTo': str(year),
                                'registrationTypes': 'normal', # ---------- Static
                                'addressTypes': str(par_house_type),
                                'sold': 'true', # -------------------------- Static
                                'sortAscending': 'false', # --------------- Static
                                'sortBy': 'soldDate' # -------------------- Static
                            }
                        else:
                            parameters = {
                                'municipalities': str(municipal),
                                'per_page': '20', # ----------------------- static 
                                'page': '1', #----------------------------- static
                                'priceMax': str(par_price_max),
                                'priceMin': str(par_price_min),
                                'yearSoldFrom': str(year),
                                'yearSoldTo': str(year),
                                'registrationTypes': 'normal', # ---------- Static
                                'addressTypes': str(par_house_type),
                                'sold': 'true', # -------------------------- Static
                                'sortAscending': 'false', # --------------- Static
                                'sortBy': 'soldDate' # -------------------- Static
                            }

                        #----- get header 
                        main_header = get_header()

                        #----- make HTTPS Request 
                        totalhits = make_request(session, parameters, main_header)

                        if(totalhits<10000):
                            print('----------------------------------- BINGO')
                            
                            temp_tuple = (par_municipal, par_house_type, par_year, par_price_min, par_price_max, par_size_min, par_size_max, totalhits)
                            print("------ printing the tuble:")
                            print(f'Municipal: {temp_tuple[0]}')
                            print(f'par_house_type: {temp_tuple[1]}')
                            print(f'par_year: {temp_tuple[2]}')
                            print(f'par_price_min: {temp_tuple[3]}')
                            print(f'par_price_max: {temp_tuple[4]}')
                            print(f'par_size_min: {temp_tuple[5]}')
                            print(f'par_size_max: {temp_tuple[6]}')
                            print(f'TOTALHITS: {temp_tuple[7]}')
                            
                            parameter_list.append(temp_tuple)
#                             return (temp_tuple)
                        
                        else:
                            # ----------------------------------------------------------
                            # - Total hits : Municipality + housing type + year + Price + Size !
                            # ----------------------------------------------------------

                            for size_index in range(0,len(parameter_size)):

                                # ----- set price min and max 
                                if(parameter_size[size_index] == 290): 
                                    par_size_min = parameter_size[size_index]
                                    par_size_max = 'infinity'
                                else:
                                    par_size_min = parameter_size[size_index]
                                    par_size_max = parameter_size[size_index+1]


                                print('-----------------------------------')
                                print(f'+ Municipality: {municipal}')
                                print(f'+ Housing type: {house_category}')
                                print(f'+ Year: {year}')
                                print(f'+ price minimum: {par_price_min}')
                                print(f'+ price minimum: {par_price_max}')
                                print('-----------------------------------')


                                #----- set parameter
                                if par_size_max == "infinity":
                                    parameters = {
                                        'municipalities': str(municipal),
                                        'per_page': '20', # ----------------------- static 
                                        'page': '1', #----------------------------- static
                                        'priceMax': str(par_price_max),
                                        'priceMin': str(par_price_min),
                                        'yearSoldFrom': str(year),
                                        'yearSoldTo': str(year),
                                        'registrationTypes': 'normal', # ---------- Static
                                        'addressTypes': str(par_house_type),
                                        'areaMin': str(par_size_min),
                                        'areaMax': str(par_size_max),
                                        'sold': 'true', # -------------------------- Static
                                        'sortAscending': 'false', # --------------- Static
                                        'sortBy': 'soldDate' # -------------------- Static
                                    }
                                else:
                                    parameters = {
                                        'municipalities': str(municipal),
                                        'per_page': '20', # ----------------------- static 
                                        'page': '1', #----------------------------- static
                                        'priceMax': str(par_price_max),
                                        'priceMin': str(par_price_min),
                                        'yearSoldFrom': str(year),
                                        'yearSoldTo': str(year),
                                        'registrationTypes': 'normal', # ---------- Static
                                        'addressTypes': str(par_house_type),
                                        'areaMin': str(par_size_min),
                                        'areaMax': str(par_size_max),
                                        'sold': 'true', # -------------------------- Static
                                        'sortAscending': 'false', # --------------- Static
                                        'sortBy': 'soldDate' # -------------------- Static
                                    }

                                #----- get header 
                                main_header = get_header()

                                #----- make HTTPS Request 
                                totalhits = make_request(session, parameters, main_header)

                                temp_tuple = (par_municipal, par_house_type, par_year, par_price_min, par_price_max, par_size_min, par_size_max, totalhits)
                                print("------ printing the tuble:")
                                print(f'Municipal: {temp_tuple[0]}')
                                print(f'par_house_type: {temp_tuple[1]}')
                                print(f'par_year: {temp_tuple[2]}')
                                print(f'par_price_min: {temp_tuple[3]}')
                                print(f'par_price_max: {temp_tuple[4]}')
                                print(f'par_size_min: {temp_tuple[5]}')
                                print(f'par_size_max: {temp_tuple[6]}')
                                print(f'TOTALHITS: {temp_tuple[7]}')
                                
                                parameter_list.append(temp_tuple)
                                
#                                 return (temp_tuple)


In [ ]:
# ------------ parameter list collect all the parameters that are less then 10.000 hits
parameter_list = []

# ------------ Setting up the parameters 
parameter_year = list(range(2023,2012, -1))

parameter_housingType = ['villa','condo','terraced house']

parameter_price= list(range(0,24500000,500000)) # increment by 500.000 dkk
parameter_price.append(24900000)

parameter_size = list(range(0,300,20)) # increment by 20m^2
parameter_size.append(290)

# -------------- set up the session 
session = requests.Session()

# -------------- run the scraper 
for municipal in municipalities_list: 
    
    # get the parameters that get less than 10.000 hits
    check_hits(municipal, parameter_housingType,parameter_year,parameter_price, parameter_size,session)
    
    # append to the parameter_list 
#     parameter_list.append(parameters)
    

In [79]:
import math
# convert the list of tuples to pandas dataframe

column_names = [
    'Municipality',
    'Property_type',
    'Year',
    'Price_min',
    'Price_max',
    'Size_min',
    'Size_max',
    'totalhits'
]

df = pd.DataFrame(parameter_list, columns=column_names)

# create column "number_of_pages"
df["number_of_pages"] = df["totalhits"]/20

# Round up
for index, row in df.iterrows():
    df.at[index, 'number_of_pages'] = math.ceil(row['number_of_pages'])
    
# Convert the 'Value' column to integers
df['number_of_pages'] = df['number_of_pages'].astype(int)

df

,Municipality,Property_type,Year,Price_min,Price_max,Size_min,Size_max,totalhits,number_of_pages
0,København,villa,2023,nothing,nothing,nothing,nothing,414,21
1,København,villa,2022,nothing,nothing,nothing,nothing,426,22
2,København,villa,2021,nothing,nothing,nothing,nothing,676,34
3,København,villa,2020,nothing,nothing,nothing,nothing,750,38
4,København,villa,2019,nothing,nothing,nothing,nothing,620,31
...,...,...,...,...,...,...,...,...,...
699,Fanø,condo,nothing,nothing,nothing,nothing,nothing,36,2
700,Fanø,terraced house,nothing,nothing,nothing,nothing,nothing,238,12
701,Læsø,villa,nothing,nothing,nothing,nothing,nothing,760,38
702,Læsø,condo,nothing,nothing,nothing,nothing,nothing,31,2


In [80]:
# save frame
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boligsiden_dk\Step_1_Scrape_CSV_JSON_rows\Totalhits_parameters_for_request.csv'
df.to_csv(path,encoding="utf-8")

In [83]:
# convert table to a list of tubles 
list_of_parameters = [tuple(x) for x in df.to_records(index=False)]
len(list_of_parameters)

704

In [ ]:
list_of_parameters

# _______________________ [Step 2] __________________________

In [20]:
### read in the parameter list 
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boligsiden_dk\Step_1_Scrape_CSV_JSON_rows\Totalhits_parameters_for_request.csv'
list_of_parameters = pd.read_csv(path, encoding="utf-8")
list_of_parameters = list_of_parameters.drop(columns=['Unnamed: 0'])
# convert table to a list of tubles 
list_of_parameters = [tuple(x) for x in list_of_parameters.to_records(index=False)]
len(list_of_parameters)

704

In [21]:
list_of_parameters

[('København',
  'villa',
  '2023',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  414,
  21),
 ('København',
  'villa',
  '2022',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  426,
  22),
 ('København',
  'villa',
  '2021',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  676,
  34),
 ('København',
  'villa',
  '2020',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  750,
  38),
 ('København',
  'villa',
  '2019',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  620,
  31),
 ('København',
  'villa',
  '2018',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  664,
  34),
 ('København',
  'villa',
  '2017',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  745,
  38),
 ('København',
  'villa',
  '2016',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  726,
  37),
 ('København',
  'villa',
  '2015',
  'nothing',
  'nothing',
  'nothing',
  'nothing',
  726,
  37),
 ('København',
  'villa',
  '2014',
  'nothing',
  'nothing',
  'nothing',
  'noth

# Program: Scrape the JSON data

In [23]:
def scrape_data(parameters,session):
    
    global gagna_listinn
    
    # ---------------------
    # Get the header 
    # ---------------------
    main_header = get_header()
    
    # make the Request to the API
    response = session.get('https://api.boligsiden.dk/search/addresses', params=parameters, headers=main_header)
    
    if response.status_code == 200:
        # get all the properties from the JSON
        property_json_list = response.json()['addresses']

        # add each property JSON individually into the recordlist such that 1 index number == 1 property
        for prop in range(0,len(property_json_list)):
            
            # Construct a DataFrame from the JSON object
            df = pd.DataFrame([property_json_list[prop]])
            
            # Append the DataFrame to the list of DataFrames
            gagna_listinn.append(df)
            
    else:
        print()
        print('############################################################')
        print('------------------- BAD REQUEST, NOT 200 ------------------')
        print('############################################################')
        print()
    

# Program :start = set up the parameter 

In [24]:
def start(parameters_tuble,session):
    
    # get the number of pages to itterate
    number_of_pages = parameters_tuble[8]
    
    for page in range(1,number_of_pages+1): 
        
        # -------------------------------
        # Create the paramter dictionary 
        #--------------------------------
        parameters = {}

        #---- add municipality 
        parameters['municipalities'] = str(parameters_tuble[0])
        
        print('::::::::::::::::::::::::::::::::::')
        print(f'{parameters_tuble[0]}')
        print('::::::::::::::::::::::::::::::::::')
        
        print(f'------------------------- page: {page} out of {number_of_pages}')


        #---- add per_page 
        parameters['per_page'] = '20'
        
        #---- add page number
        parameters['page'] = str(page)
        
        #---- add price MAX 
        if(str(parameters_tuble[4]) != 'nothing'):
            parameters['priceMax'] = str(parameters_tuble[4])
        
        #---- add price MIN 
        if(str(parameters_tuble[3]) != 'nothing'):
            parameters['priceMin'] = str(parameters_tuble[3])
        
        #---- add year:
        if(str(parameters_tuble[2]) != 'nothing'):
            parameters['yearSoldFrom'] = str(parameters_tuble[2])
        if(str(parameters_tuble[2]) != 'nothing'):
            parameters['yearSoldTo'] = str(parameters_tuble[2])
        
        #---- add NORMAL registration of sale
        parameters['registrationTypes'] = 'normal'
        
        #---- add property type (addresstype)
        parameters['addressTypes'] = str(parameters_tuble[1])

        #----- add minimum size parameter (areaMin)
        if(str(parameters_tuble[5]) != 'nothing'):
            parameters['areaMin'] = str(parameters_tuble[5])
            
        #----- add Maximum size parameter (areaMin)
        if(str(parameters_tuble[6]) != 'nothing'):
            parameters['areaMax'] = str(parameters_tuble[6])
            
        #---- add sold:true 
        parameters['sold'] = 'true'
        #---- add sold:true 
        parameters['sortAscending'] = 'false'
        #---- add sold:true 
        parameters['sortBy'] = 'soldDate'
        
#         print(f'-------------------- TOTAL NUMBER OF PAGES: {number_of_pages}')
#         print(parameters)
        print()
        
        #########################
        # Start scraping
        #########################
        scrape_data(parameters,session)
    

# Main function: set all up and run baby 

In [ ]:

# set up a session
session = requests.Session()

counter = 0 

for row in list_of_parameters:
    counter +=1


    # keep track of each property for each page then save to csv
    gagna_listinn = []

    print(f'################################ Row {counter} of 704 ###################')

    # scrape: 
    start(row,session)

    # Concatenate the list of DataFrames into a single DataFrame
    final_df = pd.concat(gagna_listinn, ignore_index=True)

    # Save the final DataFrame to a single CSV file
    base = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boligsiden_dk\Step_1_Scrape_CSV_JSON_rows\data'

    municipal = row[0]
    parameter_number = counter
    setup = fr'\Boligsiden_properties_{municipal}_parameterRow_{parameter_number}.csv'

    new_path = base + setup

    final_df.to_csv(new_path, index=False, mode='a', encoding='utf-8')#, header=False)

#     with open(path, mode='w', newline='', encoding='utf-8') as file:
#         writer = csv.writer(file, delimiter=',')